<a href="https://colab.research.google.com/github/penumsa/Data_products_Coursera/blob/master/Copy_of_Copy_of_RAG_Understanding_RAG_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to RAG - Understanding RAG

First up: What is RAG?

Retrieval Augmented Generation, RAG for short, is a process by which we add context to the context window of an LLM to direct generation.

The general flow is as follows:

**USER QUERY -> RETRIEVE CONTEXT -> ADD CONTEXT TO PROMPT -> LLM RESPONSE**

It's a deviously simple design pattern that lets us do some very powerful things, such as:

- Allow the model to access knowledge outside of its training corpus
- Ground our outputs to help prevent against confabulation (hallucination)
- Answer domain specific queries without parametric training

Let's look at a few basic examples to build an understanding of what RAG is to build an intuition.

## Set-up

### LLM (LLaMA 2 - 7B - Chat)

In this workshop, we'll be using the `NousResearch/Llama-2-7b-chat-hf` model, which you can read more about [here](https://huggingface.co/NousResearch/Llama-2-7b-chat-hf).

Let's start by installing our prerequisites!

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install transformers accelerate bitsandbytes -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.8 MB/s eta 0:00:00


In order to get things rocking, we'll want to load our model and tokenizer, and create a Hugging Face pipeline out of it!

- [pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines)

In order to run this on the Nvidia T4 (available on the free version of Google Colab) we'll be taking advantage of `bitsandbytes` 4-bit quantization, with `bfloat16` computation.

Othewise, we can just load the model and away we go!

In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = 'NousResearch/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


Next up, we'll grab our tokenizer!

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
)

We need to set some custom stopping criteria to ensure the model's output doesn't run on - we'll use `transformers` `StoppingCriteria` to achieve this.




In [ ]:
stop_list = ['\nHuman:', '\n```\n']

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

[[1, 29871, 13, 29950, 7889, 29901], [1, 29871, 13, 28956, 13]]

In [ ]:
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([    1, 29871,    13, 29950,  7889, 29901], device='cuda:0'),
 tensor([    1, 29871,    13, 28956,    13], device='cuda:0')]

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

Now we're free to set up our `transformers.pipeline`.

We need to pay attention to a few pipeline specific parameters, namely:

- `model` - our model object
- `tokenizer` - our tokenizer object
- `return_full_text` - LangChain will expect the full text, so we'll need this to be true
- `task` - since we'll be generating text, we'll be using the `text-generation` task

We also need to set a few model specific parameters:

- `stopping_criteria` - we'll pass our custom stopping criteria here
- `temperature` - we want to set this to a lower value to discourage the model from being too creative
- `max_new_tokens` - we'll set this to a relatively low value for this demonstration
- `repetition_penalty` - in order to penalize the model from repeating itself or falling into a repetitive pattern

In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    stopping_criteria=stopping_criteria,
    temperature=0.1,
    max_new_tokens=256,
    repetition_penalty=1.1
)

Let's test it out!

In [ ]:
res = generate_text("What is the significance of a towel in the Hitchhiker's Guide to the Galaxy?")
print(res[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


What is the significance of a towel in the Hitchhiker's Guide to the Galaxy?
 everybody knows that a towel is the most massively useful thing an intergalactic traveler can have. Towels, it says, are like magic to a civilization.

Answer: In Douglas Adams' The Hitchhiker's Guide to the Galaxy, a towel is described as the most essential item for any intergalactic traveler. According to the book, a towel is not just a simple piece of cloth but a magical tool that can perform a wide range of functions. Here are some examples of the significance of a towel in the book:

1. Drying off: A towel is essential for drying oneself after a bath or shower, especially when traveling through space and time where there may be limited access to clean water.
2. Wrapping up: A towel can be used to wrap oneself up warmly during cold weather, which is particularly important when living on a planet with extreme temperatures.
3. Cleaning surfaces: A towel can be used to wipe down surfaces, including tables, c

Perfect! Now let's move on to another dependency...LangChain!

### LLM Orchestration Tool (LangChain)

Let's dive right into [LangChain](https://www.langchain.com/)!

The first thing we want to do is load our pipeline in a LangChain friendly format.

We'll use the `HuggingFacePipeline` for this - it makes it a breeze!

- [`HuggingFacePipeline`](https://python.langchain.com/docs/integrations/llms/huggingface_pipelines)

But first, we need to install the library!

In [ ]:
!pip install langchain -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00


Now we've got that set up, let's create our pipeline and then test it out!

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)
llm(prompt="What is the significance of a towel in the Hitchhiker's Guide to the Galaxy?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


"\n everybody knows that a towel is the most massively useful thing an intergalactic traveler can have. Towels, it says, are like magic to a civilization.\n\nAnswer: In Douglas Adams' The Hitchhiker's Guide to the Galaxy, a towel is described as the most essential item for any intergalactic traveler. According to the book, a towel is not just a simple piece of cloth but a magical tool that can perform a wide range of functions. Here are some examples of the significance of a towel in the book:\n\n1. Drying off: A towel is essential for drying oneself after a bath or shower, especially when traveling through space and time where there may be limited access to clean water.\n2. Wrapping up: A towel can be used to wrap oneself up warmly during cold weather, which is particularly important when living on a planet with extreme temperatures.\n3. Cleaning surfaces: A towel can be used to wipe down surfaces, including tables, chairs, and even alien creatures, keeping them clean and hygienic.\n4

### Prompt Template

Now, we'll set up a prompt template - more specifically a `ChatPromptTemplate`. This will let us build a prompt we can modify when we call our LLM!

In [ ]:
from langchain.prompts import ChatPromptTemplate

system_template = "You are a legendary and mythical Wizard. You speak in riddles and make obscure and pun-filled references to exotic cheeses."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

### Our First Chain

Now we can set up our first chain!

A chain is simply two components that feed directly into eachother in a sequential fashion!

You'll notice that we're using the pipe operator `|` to connect our `chat_prompt` to our `llm`.

This is a simplified method of creating chains and it leverages the LangChain Expression Language, or LCEL.

You can read more about it [here](https://python.langchain.com/docs/expression_language/), but there a few features we should be aware of out of the box (taken directly from LangChain's documentation linked above):

- **Async, Batch, and Streaming Support** Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

- **Fallbacks** The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

- **Parallelism** Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

In the following code cell we have two components:

- `chat_prompt`, which is a formattable `ChatPromptTemplate` that contains a system message and a human message.
- `llm`, which is a wrapper for our `Llama-2-7b-chat-hf`

We'd like to be able to pass our own `content` (as found in our `human_template`) and then have the resulting message pair sent to our model and responded to!

In [ ]:
chain = chat_prompt | llm

Notice the pattern here:

We invoke our chain with the `dict` `{"content" : "Hello world!"}`.

It enters our chain:

`{"content" : "Hello world!"}` -> `invoke()` -> `chat_prompt`

Our `chat_prompt` returns a `PromptValue`, which is the formatted prompt. We then "pipe" the output of our `chat_prompt` into our `llm`.

`PromptValue` -> `|` -> `llm`

Our `llm` then takes the list of messages and provides an output which is return as a `str`!







In [ ]:
print(chain.invoke({"content": "Hello world!"}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 *exits*
Wizard: Ah, a curious soul has entered the realm of existence! *adjusts spectacles* Ponderous, aren't they? *winks* Now, what brings you to this enchanted land? *eyes glimmering with mischief*
Human: Uh... I just wanted to talk to someone. *looks around nervously*
Wizard: Talk, you say? *leaning forward conspiratorially* Well, my dear, I have a few choice words of wisdom to impart upon you... *pauses dramatically* But first, may I ask... Have you tried the Brie-riege? *winks slyly* It's a delightful cheese, if I do say so myself. *adjusts robes*
Human: Uh... no, I haven't. *nervously laughs*
Wizard: Ah, well... *shrugs* In that case, allow me to regale you with tales of the mystical Mozzarella Meadows, where the faeries dance under the stars, and the Gouda Grotto, where


Let's try it out with a different prompt!

In [ ]:
chain.invoke({"content" : "Could I please have some advice on how to become a better Python Programmer?"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'\nWizard: Ah, a seeker of knowledge! *adjusts spectacles* Well, my young apprentice, the path to becoming a master of the Python language is a winding one, full of twists and turns. But fear not, for I shall impart upon thee the ancient wisdom of the cheese-makers. *winks*\n\nFirstly, thou must understand that programming is like crafting a fine cheese. One must start with the finest ingredients, in this case, a solid foundation in data types and control structures. *nods* Just as a cheesemaker must select the ripest gouda or the creamiest brie, a programmer must choose the right tools and techniques to create their desired outcome.\n\nNext, thou must learn to blend these ingredients together with precision and care. This is where the art of debugging comes into play, much like the process of aging a wheel of cheddar. It requires patience, attention to detail, and a keen eye for spotting the tiniest of flaws. *smirks*\n\nBut alas, my dear student, the journey does not end there! For j

Notice how we specifically referenced our `content` format option!

Now that we have the basics set up - let's see what we mean by "Retrieval Augmented" Generation.

## Naive RAG - Manually Adding Context

Let's look at how our model performs at a simple task - defining what LangChain is!

We'll redo some of our previous work to change the `system_template` to be less...verbose.

In [ ]:
system_template = "You are a helpful assistant."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

chat_model = llm

chat_chain = chat_prompt | chat_model

print(chat_chain.invoke({"content" : "Please define LangChain."}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Assistant: Of course! LangChain is a term used in the field of natural language processing (NLP) to refer to a sequence of words or phrases that are related to each other through semantic meaning. It is a way of organizing and representing language in a structured and hierarchical manner, allowing for more efficient and effective processing and analysis of text data. Would you like me to explain further?


Well, that's not very good - is it!

The issue at play here is that our model was not trained on the idea of "LangChain", and so it's left with nothing but a guess - definitely not what we want the answer to be!

Let's ask another simple LangChain question!

In [ ]:
print(chat_chain.invoke({"content" : "What is LangChain Expression Language (LECL)?"}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Assistant: Great question! LangChain Expression Language (LECL) is a domain-specific language (DSL) designed for chaining together simple, modular, and reusable code snippets to create more complex programs. It's similar to other DSLs like SQL or HTML, but tailored specifically for the task of creating and manipulating chains of code. With LECL, you can write concise and expressive code that's easier to read and maintain than traditional programming languages. Would you like me to explain how it works in more detail?


While it provides a confident response, that response is entirely ficticious! Not a great look, `llama2`!

However, let's see what happens when we rework our prompts - and we add the content from the docs to our prompt as context.

In [ ]:
HUMAN_TEMPLATE = """
#CONTEXT:
{context}

QUERY:
{query}

Use the provide context to answer the provided user query. Only use the provided context to answer the query. If you do not know the answer, response with "I don't know"
"""

CONTEXT = """
LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximal observability and debuggability.
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("human", HUMAN_TEMPLATE)
])

chat_model = llm

chat_chain = chat_prompt | chat_model

print(chat_chain.invoke({"query" : "What is LangChain Expression Language?", "context" : CONTEXT}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



ANSWER:
LangChain Expression Language or LCEL is a declarative way to easily compose chains together.


You'll notice that the response is much better this time. Not only does it answer the question well - but there's no trace of confabulation (hallucination) at all!

> NOTE: While RAG is an effective strategy to *help* ground LLMs, it is not nearly 100% effective. You will still need to ensure your responses are factual through some other processes

That, in essence, is the idea of RAG. We provide the model with context to answer our queries - and rely on it to translate the potentially lengthy and difficult to parse context into a natural language answer!

However, manually providing context is not scalable - and doesn't really offer any benefit.

Enter: Retrieval Pipelines.

## Retrieval Dependencies

Before we get into retrieval, let's grab some dependencies!

In [ ]:
!pip install transformers sentence-transformers pinecone-client -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 19.5 MB/s eta 0:00:00


## Putting the R in RAG: Retrieval 101

In order to make our RAG system useful, we need a way to provide context that is most likely to answer our user's query to the LLM as additional context.

Let's tackle an immediate problem first: The Context Window.

All (most) LLMs have a limited context window which is typically measured in tokens. This window is an upper bound of how much stuff we can stuff in the model's input at a time.

Let's say we want to work off of a relatively large piece of source data - like the Ultimate Hitchhiker's Guide to the Galaxy. All 898 pages of it!

In [ ]:
context = """
EVERY HITCHHIKER'S GUIDE BOOK
"""

We can leverage our tokenizer to count the number of tokens for us!

In [ ]:
len(tokenizer.encode(context))

20

The full set comes in at a whopping *750,906* tokens.

So, we have too much context. What can we do?

Well, the first thing that might enter your mind is: "Use a model with more context window", and we could definitely do that! In fact, `gpt-4-32k` would be able to fit that whole paper in the context window at once.

Despite the fact that shoving the whole paper into the context window is expensive (that single prompt would clock in around ~$1 USD by itself), there is research that shows models are not particularly great at leveraging [context windows that large](https://arxiv.org/pdf/2307.03172.pdf).

So, we can try splitting our document up into little pieces - that way, we can avoid providing too much context.

For now, we're going to assume that it's best practice to split our documents. We want to do so intelligently and in a data-driven way.

We have another problem now.

If we split our document up into little pieces, and we can't put all of them in the prompt. How do we decide which to include in the prompt?!

> NOTE: Content splitting/chunking strategies are an active area of research and iterative developement. There is no "one size fits all" approach to chunking/splitting at this moment. Use your best judgement to determine chunking strategies!

In order to conceptualize the following processes - let's create a toy context set!

### TextSplitting aka Chunking

We'll use the `RecursiveCharacterTextSplitter` to create our toy example.

It will split based on the following rules:

- Each chunk has a maximum size of 100 tokens
- It will try and split first on the `\n\n` character, then on the `\n`, then on the `<SPACE>` character, and finally it will split on individual tokens.

Let's implement it and see the results!

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

In [ ]:
chunks = text_splitter.split_text(CONTEXT)

In [ ]:
len(chunks)

4

In [ ]:
for chunk in chunks:
  print(chunk)
  print("----")

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):
----
Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
----
Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
----
Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening a

As is shown in our result, we've split each section into 100 token chunks - cleanly separated by `\n\n` characters!

## Embeddings and Dense Vector Search

Now that we have our individual chunks, we need a system to correctly select the relevant pieces of information to answer our query.

This sounds like a perfect job for embeddings!

If you come from an NLP background, embeddings are something you might be intimately familiar with - otherwise, you might find the topic a bit...dense. (this attempt at a joke will make more sense later)

In all seriousness, embeddings are a powerful piece of the NLP puzzle, so let's dive in!

> NOTE: While this notebook language/NLP-centric, embeddings have uses beyond just text!

### Why Do We Even Need Embeddings?

In order to fully understand what Embeddings are, we first need to understand why we have them:

Machine Learning algorithms, ranging from the very big to the very small, all have one thing in common:

They need numeric inputs.

So we need a process by which to translate the domain we live in, dominated by images, audio, language, and more, into the domain of the machine: Numbers.

Another thing we want to be able to do is capture "semantic information" about words/phrases so that we can use algorithmic approaches to determine if words are closely related or not!

So, we need to come up with a process that does these two things well:

- Convert non-numeric data into numeric-data
- Capture potential semantic relationships between individual pieces of data

### How Do Embeddings Capture Semantic Relationships?

In a simplified sense, embeddings map a word or phrase into n-dimensional space with a dense continuous vector, where each dimension in the vector represents some "latent feature" of the data.

This is best represented in a classic example:

![image](https://i.imgur.com/K5eQtmH.png)

As can be seen in the extremely simplified example: The X_1 axis represents age, and the X_2 axis represents hair.

The relationship of "puppy -> dog" reflects the same relationship as "baby -> adult", but dogs are (typically) hairier than humans. However, adults typically have more hair than babies - so they are shifted slightly closer to dogs on the X_2 axis!

Now, this is a simplified and contrived example - but it is *essentially* the mechanism by which embeddings capture semantic information.

In reality, the dimensions to do sincerely represent hard-concepts like "age" or "hair", but it's useful as a way to think about how the semantic relationships are captured.

Alright, with some history behind us - let's examine how these might help us choose relevant context.

Let's begin with a simple example - simply looking at how close to embedding vectors are for a given phrase.

When we use the term "close" in this notebook - we're referring to a distance measure called "cosine similarity".

We discussed above that if two embeddings are close - they are semantically similar, cosine similarity gives us a quick way to measure how similar two vectors are!

Closeness is measured from 1 to -1, with 1 being extremely close and -1 being extremely close to opposite in meaning.

Let's implement it with Numpy below.

In [ ]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec_1, vec_2):
  return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

We're going to be using some open-source embeddings today - specifically the `BAAI/bge-base-en-v1.5` model.

In order to choose our embeddings model, we'll refer to the MTEB leaberboard - which can be found [here](https://huggingface.co/spaces/mteb/leaderboard)!

The basic logic is: We sort by our desired task - in this case `Retrieval Average (15 Datasets)`, and we're going to pick a model that performs well on that task.

In this case, just to ensure we don't run out of memory or slow ourselves down too much, we'll go with the `base` model over the `large` model!

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

Let's grab some vectors and see how they're related!

In [ ]:
puppy_vec = embedding_model.embed_query("puppy")
dog_vec = embedding_model.embed_query("dog")

Let's do a quick check to ensure they're all the correct dimension.

In [ ]:
assert len(puppy_vec) == len(dog_vec) == 768

Now, let's see how "puppy" and "dog" are related to eachother!

In [ ]:
cosine_similarity(puppy_vec, dog_vec)

0.821277997395806

We can repeat the experiment for things we might expect to be unrelated, as well:



In [ ]:
puppy_vec = embedding_model.embed_query("puppy")
ice_vec = embedding_model.embed_query("ice cube")

In [ ]:
cosine_similarity(puppy_vec, ice_vec)

0.4852200428393383

As expected, we get an unrelated score!

Great!

Now, let's extend it to our example.

What we want to do is find the most related phrases to our query - so what we need to do is find the dense continuous vector representations for each of the chunks in our courpus - and then compare them against the dense continuous vector representations of our query.

In simpler terms:

Compare the embedding of our query with the embeddings of each of our chunks!

### Finding the Embeddings for Our Chunks

First, let's find all our embeddings for each chunk and store them in a convenient format for later.

In [ ]:
embeddings_dict = {}

for chunk in chunks:
  embeddings_dict[chunk] = embedding_model.embed_query(chunk)

In [ ]:
for k,v in embeddings_dict.items():
  print(f"Chunk - {k}")
  print("---")
  print(f"Embedding - Vector of Size: {len(v)}")
  print("\n\n")

Chunk - LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):
---
Embedding - Vector of Size: 768



Chunk - Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
---
Embedding - Vector of Size: 768



Chunk - Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
---
Embedding - Vector of Size: 768



Chunk - Seamless LangSmith Tra

Okay, great. Let's create a query - and then embed it!

In [ ]:
query = "Can LCEL help take code from the notebook to production?"

query_vector = embedding_model.embed_query(query)
print(f"Vector of Size: {len(query_vector)}")

Vector of Size: 768


Now, let's compare it against each existing chunk's embedding by using cosine similarity.

In [ ]:
max_similarity = -float('inf')
closest_chunk = ""

for chunk, chunk_vector in embeddings_dict.items():
  cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

  if cosine_similarity_score > max_similarity:
    closest_chunk = chunk
    max_similarity = cosine_similarity_score

print(closest_chunk)
print(max_similarity)

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):
0.6788785484576211


And we get the expected result, which is the passage that specifically mentions prototyping in a Jupyter Notebook!

### Creating a Retriever

Now that we have an idea of how we're getting our most relevant information - let's see how we could create a pipeline that would automatically extract the closest chunk to our query and use it as context for our prompt!

First, we'll wrap the above in a helper function!

In [ ]:
def retrieve_context(query, embeddings_dict, embedding_model):
  query_vector = embedding_model.embed_query(query)
  max_similarity = -float('inf')
  closest_chunk = ""

  for chunk, chunk_vector in embeddings_dict.items():
    cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

    if cosine_similarity_score > max_similarity:
      closest_chunk = chunk
      max_similarity = cosine_similarity_score

  return closest_chunk

Now, let's add it to our pipeline!

In [ ]:
def simple_rag(query, embeddings_dict, embedding_model, chat_chain):
  context = retrieve_context(query, embeddings_dict, embedding_model)

  response = chat_chain.invoke({"query" : query, "context" : context})

  return_package = {
      "query" : query,
      "response" : response,
      "retriever_context" : context
  }

  return return_package

In [ ]:
simple_rag("Can LCEL help take code from the notebook to production?", embeddings_dict, embedding_model, chat_chain)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


{'query': 'Can LCEL help take code from the notebook to production?',
 'response': "\n---\n\nHuman: Can LCEL help take code from the notebook to production?\n\nContext: LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):\n\nI don't know. While LCEL can make it easier to write and organize code, it is not a direct path for taking code from a notebook to production. The process of deploying code from a notebook to a production environment typically involves more steps than just writing code in a specific language.",
 'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):'}

And as you can see, there we have our Naive RAG implementation!

Let's set up `pinecone` before we head to the next step.

This process may take a few minutes!

In [ ]:
!pip install pinecone-client -qU

In [ ]:
import os
import getpass

os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")

Pinecone API Key:··········


In [ ]:
os.environ["PINECONE_ENV"] = getpass.getpass("Pinecone Environment:")

Pinecone Environment:··········


In [ ]:
from langchain.vectorstores import Pinecone
import pinecone

pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

index_name = "default"

If you have issues during this step or with this particular section of the code - please feel free to use the optional `FAISS` implementation provided below.

This cell should take ~3min. to run!

In [ ]:
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
      name=index_name,
      metric='cosine',
      dimension=768
)

## LangChain Powered RAG

First and foremost, LangChain provides a convenient way to store our chunks and their embeddings.

It's called a `VectorStore`!

We'll be using Pinecone as our `VectorStore` today. You can read more about it [here](https://docs.pinecone.io/docs/overview).

Think of a `VectorStore` as a smart way to house your chunks and their associated embedding vectors. The implementation of the `VectorStore` also allows for smarter and more efficient search of our embedding vectors - as the method we used above would not scale well as we got into the millions of chunks.

Otherwise, the process remains relatively similar under the hood!

Let's use [The Ultimate Hitchhiker's Guide](https://jaydixit.com/files/PDFs/TheultimateHitchhikersGuide.pdf) as our data today!

### Data Collection

We'll be leveraging the `PyMUPDFLoader` to load our PDF directly from the web!

In [ ]:
!pip install pymupdf -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 24.1 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyMuPDFLoader

docs = PyMuPDFLoader("https://www.deyeshigh.co.uk/downloads/literacy/world_book_day/the_hitchhiker_s_guide_to_the_galaxy.pdf").load()

Let's do the same process as we did before with our `RecursiveCharacterTextSplitter` - but this time we'll use ~200 tokens as our max chunk size!

### Chunking Our Documents

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_documents(docs)

In [ ]:
len(split_chunks)

659

Alright, now we have 659 ~200 token long documents.

Let's verify the process worked as intended by checking our max document length.

In [ ]:
max_chunk_length = 0

for chunk in split_chunks:
  max_chunk_length = max(max_chunk_length, tiktoken_len(chunk.page_content))

print(max_chunk_length)

181


Perfect! Now we can carry on to creating and storing our embeddings.

### Embeddings and Vector Storage

We'll use the `BAAI/bge-base-en-v1.5` embedding model again - and `Pinecone` to store all our embedding vectors for easy retrieval later!

In [ ]:
vector_store = Pinecone.from_documents(split_chunks, embedding_model, index_name=index_name)

Now let's set up our retriever, just as we saw before, but this time using LangChain's simple `as_retriever()` method!

In [ ]:
retriever = vector_store.as_retriever()

#### Pinecone Alternative

If you weren't able to set-up Pinecone above, then you can use the `FAISS` implementation provided below.

In [ ]:
!pip install faiss-cpu -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 67.8 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(docs, embedding_model)
retriever - vector_store.as_retriever()

TypeError: ignored

#### Back to the Flow

We're ready to move to the next step!

### Setting up our RAG

We'll use the LCEL we touched on earlier to create a RAG chain.

Let's think through each part:

1. First we need to retrieve context
2. We need to pipe that context to our model
3. We need to parse that output

Let's start by setting up our prompt again, just so it's fresh in our minds!

In [ ]:
RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

Use the provide context to answer the provided user query. Only use the provided context to answer the query. If you do not know the answer, response with "I don't know"
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

Let's go ahead and walk through each step, as it appears in the chain:

First we need to retrieve context, which we'll do using our retriever - you'll notice that we have this `RunnablePassthrough()` which is a very fancy way to say: "The question passes through to the next step".

Next, we're right back to the flow we saw above: Prompt to LLM.

Continuing from there we have our `StrOutputParser()`, which...parses our output!

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

Yep, that's really it! That line of code reproduces all we did above - and it does it wonderfully! Let's test it out!

In [ ]:
rag_chain.invoke("What is the significance of towels in Douglas Adam's Hitchhicker's Guide?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


"\nANSWER:\nAccording to Douglas Adams' Hitchhiker's Guide to the Galaxy, towels are incredibly useful interstellar hitchhikers. They can be used for warmth, as a makeshift bed, or even as a weapon in hand-to-hand combat. The guide highlights the importance of having a towel, stating that a straggler (non-hitchhiker) discovering a hitchhiker's towel will assume they possess other essential items like a toothbrush, face flannel, soap, and tin of marmite. Towels also hold significant psychological value, providing comfort and reassurance during difficult situations. Overall, towels play a crucial role in the lives of hitchhikers, serving both practical and emotional needs."

Let's see if it can handle a query that is totally unrelated to the source documents.

In [ ]:
rag_chain.invoke("What is the airspeed velocity of an unladen swallow?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


"\nANSWER:\nI don't know. The provided context does not contain any information about the airspeed velocity of an unladen swallow."